In [ ]:
import itertools
from matplotlib import pyplot as plt
import seaborn as sns
import math
import scipy
import numpy as np
import pandas as pd
from pathlib import Path
from zipfile import ZipFile
import copy
import scipy.stats as st
import statsmodels.api as sm
import seaborn as sns
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

Выборка состоит из трех обзектов: X_train, Y_train, X_test

Y_train - изображение (пиксель кодируется черно-белой компонентой изображения), X_train - признаки, соответствующие этому изображению. (элемент X[i,j] соответствует набору признаков для пикселя Y[i,j]).

Требуется:

- Провести отбор наиболее значимых признаков и построить регрессию X->Y
- Проинтерпетировать признаки (каждый признак является функцией, возможно нелинейной, от значения пикселя)
- Получить изображение по X_test (оцениваться будет качество полученного изображения. Ожидается, что регрессия на X_train, Y_train даст R^2>0.85).

In [ ]:
if Path('regression').is_dir() is False and Path('regression.zip').exists():
    with ZipFile(Path('regression.zip')) as f:
        f.extractall('regression')
x_test = np.load('regression/regression_x_test.npy')
x_train = np.load('regression/regression_x_train.npy')
y_train = np.load('regression/regression_y_train.npy')

In [ ]:
x_train.shape, y_train.shape, x_test.shape

In [ ]:
test_img_shape =  x_test.shape

In [ ]:
np.all(y_train >= 0), np.all(y_train <= 1)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(y_train)  
plt.show()

In [ ]:
x_train = x_train.reshape(-1, x_train.shape[-1])
x_test = x_test.reshape(-1, x_test.shape[-1])
y_train = y_train.reshape(-1, 1)

x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)
x_test = pd.DataFrame(x_test)

In [ ]:
x_train['bias'] = 1.0
x_test['bias'] = 1.0

Линейная регрессия без отбора признаков:

In [ ]:
pure_lin_reg = OLS(y_train, x_train).fit()
pure_lin_reg.summary()

Cделать отбор признаков с помощью значения p_value нельзя

### Сделаем отбор признаков

In [ ]:
data = copy.copy(x_train)
data['target'] = y_train[0]
data.columns = list(map(lambda i: f'feature_{i}', range(data.shape[1]-2))) + ['bias', 'target']

In [ ]:
Построим одномерные модели и оценим их значимость


In [ ]:
features = list(data.columns[:-2])
pvalues = dict()

for f in features:
    lin_reg_one = sm.GLM.from_formula(f'target~{f}', data=data).fit()
    pvalues[f] = lin_reg_one.pvalues[f]
print('pvalues: ', pvalues)

In [ ]:
filtered_features = list(filter(lambda f: pvalues[f] < 0.05, features))
print(filtered_features)
print(f'Число значимых признаков: {len(filtered_features)}')

In [ ]:
formula = formula = f"target ~ {' + '.join(filtered_features)}"
lin_reg_filtered = sm.GLM.from_formula(formula, data=data).fit()
lin_reg_filtered.summary2()

In [ ]:
wald_description = '='.join(filtered_features+[''])
print(f"Pvalue: {lin_reg_filtered.wald_test(wald_description+'0').pvalue}")

Такой способ не подходит

Попробуем проанализировать, какие признаки можно убрать

In [ ]:
plt.figure(figsize=(20,15))
corr = x_train.corr()
sns.heatmap(corr, annot=True)
plt.show()

Заметим, что признаки 30-34 - константы

In [ ]:
x_train.loc[:, 30].unique()

Оставим нескоррелированные признаки

In [ ]:
rest = [0,1,2,3,4,22,26,'bias']
x_train = x_train[rest]
x_test = x_test[rest]

In [ ]:
plt.figure()
corr = x_train.corr()
sns.heatmap(corr, annot=True)
plt.show()

Призаки 22 и 26 также имеют большой коэффициент корреляции

Построим pairplot, чтобы проверить, насколько они зависимы от 3 и 4

In [ ]:
data = copy.copy(x_train)
data['target'] = y_train[0]
_ = sns.pairplot(data[:100])

Они действительно сильно зависят, поэтому их тоже следует убрать

In [ ]:
x_train.drop(columns = [22,26], inplace=True)
x_test.drop(columns = [22,26], inplace=True)
data = copy.copy(x_train)
data['target'] = y_train[0]
_ = sns.pairplot(data[:50])

In [ ]:
data.columns = list(map(lambda i: f'feature_{i}', range(data.shape[1]-2))) + ['bias', 'target']


In [ ]:
formula = f"target ~ {' + '.join(data.columns[:-1])}"
lin_reg_1 = sm.OLS.from_formula(formula, data=data).fit()
lin_reg_1.summary()

Отношение объясненной дисперсии не очень высокое, меньше чем оное без отбора признаков

### Добавим межфакторные взаимодействия

In [ ]:
for i, f1 in enumerate(data.columns[:-1]):
    for f2 in data.columns[i+1:-1]: 
        model = sm.GLM.from_formula(f"{formula} + {f1} : {f2}", data=data).fit()               
        if model.pvalues[f'{f1}:{f2}'] < 0.05:
            print (f1, f2, model.bic)

Оставим feature_1:feature_3 и feature_2:feature_3

In [ ]:
lin_reg_2 = sm.OLS.from_formula(f"{formula} + feature_1:feature_3 + feature_2:feature_3", data=data).fit() 
lin_reg_2.summary()

In [ ]:
lin_reg_2.wald_test('feature_1:feature_3 = feature_2:feature_3 = 0')

In [ ]:
plt.hist(lin_reg_2.resid, bins = 100)
plt.show()

Пока ошибка выглядит не совсем нормально, поэтому добавим ещё межфакторных взаимодействий 

In [ ]:
features = data.columns[:-2]
f_f = list(itertools.chain.from_iterable([[f'{f1}:{f2}' for f2 in features[i+1:]] 
                                          for i, f1 in enumerate(features)]))

In [ ]:
lin_reg_3 = sm.OLS.from_formula(f"{formula}+{'+'.join(f_f)}", data=data).fit() 
lin_reg_3.summary()

Теперь отношение объясненной дисперсии хорошее

In [ ]:
lin_reg_3.wald_test(f"{' = '.join(f_f)} = 0")

In [ ]:
plt.hist(lin_reg_3.resid, bins = 100)
plt.show()

### Восстановим картинку

In [ ]:
x_test.columns = list(map(lambda i: f'feature_{i}', x_test.columns[:-1])) + ['bias']

In [ ]:
prediction = lin_reg_3.predict(x_test)
y_test = np.reshape(prediction.values, (test_img_shape[:-1]))
fig, ax = plt.subplots()
ax.imshow(y_test)
plt.show()